In [ ]:
!pip install diffusers transformers accelerate gradio

In [ ]:
import random
import gradio as gr
from transformers import pipeline
from diffusers import StableDiffusionPipeline
import torch

# Load LLM for filtering
generator = pipeline("text2text-generation", model="google/flan-t5-large")

# Load Stable Diffusion locally
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
pipe = pipe.to("cuda")

# Candidate word list
candidate_words = ["apple", "tiger", "car", "happiness", "mountain", "justice", "dog", "city"]

# Function to use LLM to identify if word can be represented easily in an image
def is_image_friendly(word):
    prompt = (
        f"Only respond with 'Yes' or 'No'. "
        f"Can the word '{word}' be clearly represented as a single object or scene in an image?"
    )
    response = generator(prompt, max_length=10, do_sample=False)
    answer = response[0]['generated_text'].strip().lower()
    #print(f"Word: {word}, LLM Response: '{answer}'")

    if "yes" in answer:
        return True
    elif "no" in answer:
        return False
    else:
        # print(f"Unclear response for '{word}', defaulting to YES for demo.")
        return True


# Filter out words that can't be represented by images
def get_valid_words():
    valid_words = []
    for word in candidate_words:
        if is_image_friendly(word):
            valid_words.append(word)
    return valid_words

# Function to scramble up a given word
def scramble_word(word):
    if len(set(word)) <= 1:
        return word
    word_list = list(word)
    while True:
        random.shuffle(word_list)
        scrambled = ''.join(word_list)
        if scrambled != word:
            return scrambled

# Game Logic Function for Gradio
valid_words = get_valid_words()
current_word = random.choice(valid_words)
scrambled_current_word = scramble_word(current_word)

def play_game(user_guess, show_hint):
    global current_word, scrambled_current_word, valid_words

    message = ""
    hint_image = None

    if user_guess.lower() == current_word.lower():
        message = f"✅ Correct! The word was '{current_word}'."
        valid_words.remove(current_word)
        if valid_words:
            current_word = random.choice(valid_words)
            scrambled_current_word = scramble_word(current_word)
        else:
            message += " 🎉 You've completed all words!"
            scrambled_current_word = ""
    else:
        message = "❌ Incorrect. Try again!"
        if show_hint and current_word:
            prompt = f"An image of a {current_word}"
            hint_image = pipe(prompt).images[0]

    return message, scrambled_current_word, hint_image

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# AI-Powered Word Scramble Game")
    scrambled_display = gr.Textbox(label="Scrambled Word", value=scrambled_current_word, interactive=False)
    guess = gr.Textbox(label="Your Guess")
    hint_checkbox = gr.Checkbox(label="Show Hint Image if Incorrect")
    submit_btn = gr.Button("Submit Guess")
    output_msg = gr.Textbox(label="Result")
    output_image = gr.Image(label="Hint Image")

    submit_btn.click(fn=play_game,
                     inputs=[guess, hint_checkbox],
                     outputs=[output_msg, scrambled_display, output_image])

demo.launch()